In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from open_spiel.python.examples.ubc_evaluate_policy import run_eval

from auctions.webutils import *
from collections import Counter

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
experiment = Experiment.objects.get(name='infocompare')
runs = experiment.equilibriumsolverrun_set.all()

In [4]:
def analyze_checkpoint(checkpoint):
    record = dict()
    samples = checkpoint.evaluation.samples
    welfares = np.zeros(len(samples['rewards']['0']))
    revenues = np.zeros(len(samples['rewards']['0']))
    for player in range(checkpoint.equilibrium_solver_run.game.num_players):
        player = str(player)
        rewards = pd.Series(samples['rewards'][player])
        payments = pd.Series(samples['payments'][player])
        record[f'p{player}_utility'] = rewards.mean()
        record[f'p{player}_payment'] = payments.mean()
        welfares += rewards + payments
        revenues += payments
    record['total_welfare'] = welfares.mean()
    record['total_revenue'] = revenues.mean()

    arr = np.array(samples['allocations']['0']).astype(int) # TODO: only player 0's allocation.
    c = Counter(tuple(map(tuple, arr)))
    record['common_allocations'] = c.most_common(5)
    
    return record
    # print(welfares.mean(), revenues.mean())


In [5]:
checkpoint, nash_conv = find_best_checkpoint(runs[0])

In [6]:
env_and_model = db_checkpoint_loader(checkpoint)

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/large_game_2.json
Done config parsing


In [7]:
game, policy, env, agents, game_config = env_and_model.game, env_and_model.nfsp_policies, env_and_model.env, env_and_model.agents, env_and_model.game_config


In [8]:
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree
 

In [53]:
tree = sample_game_tree(env_and_model, 20, report_freq=100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 432.22it/s]


In [54]:
# tree

In [55]:
# import json
# len(json.dumps(tree)) / 1024**2

In [56]:
json.dumps(tree)

TypeError: Object of type ndarray is not JSON serializable

In [14]:
# size of tree in MB
len(pickle.dumps(tree)) / 1024**2

0.1145162582397461

In [89]:
# size of compressed tree in MB
import bz2
len(bz2.compress(pickle.dumps(tree))) / 1024**2

1.671483039855957

In [18]:
display(tree[0]['mean_outcomes'])

{'reward': 115.47, 'payment': 251.53, 'allocation': 1.37, 'length': 5.76}

In [19]:
display(tree[0]['children'].keys())

dict_keys(['Current player: 0\nPlayer 0\nValues:120, 80, 30\n Budget: 500\n', 'Current player: 0\nPlayer 0\nValues:100, 70, 50\n Budget: 500\n'])

In [20]:
display(tree[0]['children']['Current player: 0\nPlayer 0\nValues:100, 70, 50\n Budget: 500\n']['mean_outcomes'])

{'reward': 93.81, 'payment': 238.19, 'allocation': 1.45, 'length': 6.17}

In [21]:
display(tree[0]['children']['Current player: 0\nPlayer 0\nValues:120, 80, 30\n Budget: 500\n']['mean_outcomes'])

{'reward': 127.13, 'payment': 258.72, 'allocation': 1.33, 'length': 5.54}

In [72]:
display(tree[0]['children']['Current player: 0\nPlayer 0\nValues:120, 80, 30\n Budget: 500\n']['children'].keys())

dict_keys(['Bid for 3,1,0 licenses @ $275 with activity 275', 'Bid for 3,3,2 licenses @ $425 with activity 425', 'Bid for 3,3,3 licenses @ $450 with activity 450', 'Bid for 2,3,0 licenses @ $300 with activity 300', 'Bid for 3,2,0 licenses @ $325 with activity 325', 'Bid for 1,3,3 licenses @ $300 with activity 300', 'Bid for 3,0,3 licenses @ $300 with activity 300', 'Bid for 2,2,3 licenses @ $325 with activity 325', 'Bid for 2,2,1 licenses @ $275 with activity 275', 'Bid for 2,3,1 licenses @ $325 with activity 325', 'Bid for 3,0,2 licenses @ $275 with activity 275', 'Bid for 2,0,3 licenses @ $225 with activity 225', 'Bid for 2,3,3 licenses @ $375 with activity 375', 'Bid for 3,0,0 licenses @ $225 with activity 225', 'Bid for 3,3,1 licenses @ $400 with activity 400', 'Bid for 1,3,2 licenses @ $275 with activity 275', 'Bid for 2,1,3 licenses @ $275 with activity 275', 'Bid for 3,2,3 licenses @ $400 with activity 400', 'Bid for 1,1,3 licenses @ $200 with activity 200', 'Bid for 3,2,2 licen

In [81]:
player_0_type = 'Current player: 0\nPlayer 0\nValues:120, 80, 30\n Budget: 500\n'
for action in tree[0]['children'][player_0_type]['children']:
    node = tree[0]['children']['Current player: 0\nPlayer 0\nValues:120, 80, 30\n Budget: 500\n']['children'][action]
    print(f'{action}:')
    print(f'  num plays:    {node["num_plays"]}')
    print(f'  avg outcomes: {node["mean_outcomes"]}')

Bid for 3,1,0 licenses @ $275 with activity 275:
  num plays:    3972
  avg outcomes: {'reward': 137.96561578663454, 'payment': 264.2373048257972, 'allocation': 1.3267875125881168, 'length': 5.116817724068479}
Bid for 3,3,2 licenses @ $425 with activity 425:
  num plays:    595
  avg outcomes: {'reward': 119.4841857138235, 'payment': 258.80152902042164, 'allocation': 1.3529411764705883, 'length': 5.798319327731092}
Bid for 3,3,3 licenses @ $450 with activity 450:
  num plays:    217
  avg outcomes: {'reward': 117.1574164007834, 'payment': 260.99926574988297, 'allocation': 1.3625192012288787, 'length': 5.788018433179723}
Bid for 2,3,0 licenses @ $300 with activity 300:
  num plays:    1187
  avg outcomes: {'reward': 135.11437162590101, 'payment': 267.73314324807194, 'allocation': 1.3277169334456613, 'length': 5.378264532434709}
Bid for 3,2,0 licenses @ $325 with activity 325:
  num plays:    275
  avg outcomes: {'reward': 138.50049075450906, 'payment': 278.6631457796964, 'allocation': 1

In [71]:
time_step = env.reset()

In [48]:
env._state.information_state_string()

'Current player: 0\nPlayer 0\nValues:100, 70, 50\n Budget: 500\n'

In [8]:
game

turn_based_simultaneous_game(game=clock_auction(filename=large_game_2.json))

In [9]:
root = {'outcomes': [], 'children': {}}

In [10]:
root['children']['action_1'] = {'outcomes': [], 'children': {}}
root['children']['action_2'] = {'outcomes': [], 'children': {}}

In [11]:
from pprint import pprint
pprint(root)

{'children': {'action_1': {'children': {}, 'outcomes': []},
              'action_2': {'children': {}, 'outcomes': []}},
 'outcomes': []}


In [49]:
state = game.new_initial_state().child(1).child(1).child(3).child(4)

In [52]:
# dict(state.chance_outcomes())

In [53]:
state.current_player()

-4

In [38]:
state.information_state_string()

OpenSpiel exception: /apps/open_spiel/open_spiel/game_transforms/turn_based_simultaneous_game.cc:159 player >= 0
player = -4, 0 = 0


SpielError: /apps/open_spiel/open_spiel/game_transforms/turn_based_simultaneous_game.cc:159 player >= 0
player = -4, 0 = 0

In [66]:
str(state).split('\n')[-1]

'Final bids: 0, 0, 3 | 0, 1, 0'

In [56]:
state.observation_string()

OpenSpiel exception: /apps/open_spiel/open_spiel/game_transforms/turn_based_simultaneous_game.cc:200 player >= 0
player = -4, 0 = 0


SpielError: /apps/open_spiel/open_spiel/game_transforms/turn_based_simultaneous_game.cc:200 player >= 0
player = -4, 0 = 0

In [54]:
state.action_to_string(0)

'Bid for 0,0,0 licenses @ $0 with activity 0'

In [28]:
time_step = env.reset()

In [30]:
time_step.observations['info_state']

[[1.0,
  0.0,
  1.0,
  0.0,
  1.0,
  0.0,
  5.0,
  10.0,
  15.0,
  30.0,
  35.0,
  40.0,
  45.0,
  60.0,
  65.0,
  70.0,
  75.0,
  90.0,
  95.0,
  100.0,
  105.0,
  45.0,
  50.0,
  55.0,
  60.0,
  75.0,
  80.0,
  85.0,
  90.0,
  105.0,
  110.0,
  115.0,
  120.0,
  135.0,
  140.0,
  145.0,
  150.0,
  90.0,
  95.0,
  100.0,
  105.0,
  120.0,
  125.0,
  130.0,
  135.0,
  150.0,
  155.0,
  160.0,
  165.0,
  180.0,
  185.0,
  190.0,
  195.0,
  135.0,
  140.0,
  145.0,
  150.0,
  165.0,
  170.0,
  175.0,
  180.0,
  195.0,
  200.0,
  205.0,
  210.0,
  225.0,
  230.0,
  235.0,
  240.0,
  0.0,
  2.5,
  5.0,
  7.5,
  25.0,
  27.5,
  30.0,
  32.5,
  50.0,
  52.5,
  55.0,
  57.5,
  75.0,
  77.5,
  80.0,
  82.5,
  37.5,
  40.0,
  42.5,
  45.0,
  62.5,
  65.0,
  67.5,
  70.0,
  87.5,
  90.0,
  92.5,
  95.0,
  112.5,
  115.0,
  117.5,
  120.0,
  75.0,
  77.5,
  80.0,
  82.5,
  100.0,
  102.5,
  105.0,
  107.5,
  125.0,
  127.5,
  130.0,
  132.5,
  150.0,
  152.5,
  155.0,
  157.5,
  112.5,
  115.0,
 

In [23]:
eval_output = run_eval(env_and_model, 5)

In [24]:
eval_output

{'walltime': 0.012504339218139648,
 'rewards': defaultdict(list,
             {0: [160.0, 160.0, 160.0, 79.99999999999999, 160.0],
              1: [155.0, 155.0, 155.0, 120.19249999999994, 155.0]}),
 'types': defaultdict(list,
             {0: [(500.0, 120.0, 80.0, 30.0),
               (500.0, 120.0, 80.0, 30.0),
               (500.0, 120.0, 80.0, 30.0),
               (500.0, 100.0, 70.0, 50.0),
               (500.0, 120.0, 80.0, 30.0)],
              1: [(500.0, 100.0, 80.0, 60.0),
               (500.0, 100.0, 80.0, 60.0),
               (500.0, 100.0, 80.0, 60.0),
               (500.0, 100.0, 80.0, 60.0),
               (500.0, 100.0, 80.0, 60.0)]}),
 'allocations': defaultdict(list,
             {0: [[3.0, 1.0, 0.0],
               [3.0, 1.0, 0.0],
               [3.0, 1.0, 0.0],
               [2.0, 2.0, 0.0],
               [3.0, 1.0, 0.0]],
              1: [[0.0, 2.0, 3.0],
               [0.0, 2.0, 3.0],
               [0.0, 2.0, 3.0],
               [1.0, 1.0, 3.0],
   